In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
project_dir = '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23'
%cd '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src'

/content/drive/.shortcut-targets-by-id/185mvSLpCVGSaXntfPSeAocQRS6XVVq-_/IR_Project_W23/src


In [5]:
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=9eda7c432282e9f228e2528d68fde38835169bd8a5411bb7182bcb2e8303fbfb
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

### Import Libraries

In [ ]:
import os
import sys
import requests
import json
import time
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import random
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer, util, CrossEncoder, InputExample, losses, models, datasets
import faiss
import numpy as np
import pickle
from pathlib import Path
from vector_engine.vector_engine.utils import vector_search, id2details

### Loading Data

In [ ]:
# read data from the csv file (from the location it is stored)
data_dir = '../data/'
data_list = sorted(os.listdir(data_dir))
dataframe_list = []
print(data_list)

for data_file in data_list:
    if '.json' not in data_file:
        continue
    data_path = data_dir + data_file
    data_name = data_file[:-5]
    save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
    print("Data File:", data_name)
    df = pd.read_json(data_path)
    df = df.astype(str)
    rows, columns = df.shape
    dataframe_list.append(df)

df = pd.concat(dataframe_list)
df.head(10)

['LSTM_Attention', 'backup', 'cnn.json', 'podcasts.json', 'queries', 'ramsay10.json', 'reviews.json', 'summary', 'ted.json']
Data File: cnn
Data File: podcasts
Data File: ramsay10
Data File: reviews
Data File: ted


,id,text_captions,fused_captions,summary,snippet,statistics
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,"{'publishedAt': '2023-03-02T01:20:33Z', 'chann...","{'viewCount': '104381', 'likeCount': '1046', '..."
1,Kc_GMYHJNFQ,[music] Thank you hello everyone! Wonderful! W...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...","{'publishedAt': '2023-02-28T23:25:02Z', 'chann...","{'viewCount': '97761', 'likeCount': '1049', 'f..."
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"{'publishedAt': '2023-02-27T22:44:53Z', 'chann...","{'viewCount': '100789', 'likeCount': '1168', '..."
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"{'publishedAt': '2023-02-26T19:36:20Z', 'chann...","{'viewCount': '84078', 'likeCount': '759', 'fa..."
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,"{'publishedAt': '2023-02-24T00:34:44Z', 'chann...","{'viewCount': '90645', 'likeCount': '990', 'fa..."
5,1xbEbTJASoE,[music] Thank you What's up lovely people! It'...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",A massive winter storm is threatening the U.S...,"{'publishedAt': '2023-02-23T00:50:47Z', 'chann...","{'viewCount': '82775', 'likeCount': '1056', 'f..."
6,rQp4putiL_A,[music] Thank you Hello lovely people! Wonderf...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN's Brian Todd tells us about President Bid...,"{'publishedAt': '2023-02-22T00:41:45Z', 'chann...","{'viewCount': '77883', 'likeCount': '907', 'fa..."
7,94N_3jBAU_c,[music] Thank you hello everyone! Terrific! Tu...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",This week President Biden made an unexpected ...,"{'publishedAt': '2023-02-21T14:03:38Z', 'chann...","{'viewCount': '98478', 'likeCount': '807', 'fa..."
8,owDr-ra0P6I,"[music] Thank you hello lovely people, You mad...","[{'text': '[Music]', 'start': 0.0, 'duration':...",A toxic train derailment in Ohio led to the e...,"{'publishedAt': '2023-02-17T00:32:51Z', 'chann...","{'viewCount': '95155', 'likeCount': '1130', 'f..."
9,sxpPGSG7zo8,[music] Thank you hello lovely people! it's Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Inflation surged in January and by the most w...,"{'publishedAt': '2023-02-15T23:35:38Z', 'chann...","{'viewCount': '106227', 'likeCount': '1138', '..."


In [ ]:
df.describe()

,id,text_captions,fused_captions,summary,snippet,statistics
count,435,435,435,435,435,435
unique,435,435,435,435,435,435
top,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,"{'publishedAt': '2023-03-02T01:20:33Z', 'chann...","{'viewCount': '104381', 'likeCount': '1046', '..."
freq,1,1,1,1,1,1


In [ ]:
print(f"Videos: {df.id.unique().shape[0]}")
n_ids = len(df.id.unique())
id_list = df.id.unique()
id_value_list = np.arange(n_ids)
id_dict = dict(zip(id_list, id_value_list))
# id_dict = {id_list[i]: id_value_list[i] for i in range(n_ids)}
df['id_value'] = df['id'].apply(lambda x: id_dict[x])
df.head()

Videos: 435


,id,text_captions,fused_captions,summary,snippet,statistics,id_value
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,"{'publishedAt': '2023-03-02T01:20:33Z', 'chann...","{'viewCount': '104381', 'likeCount': '1046', '...",0
1,Kc_GMYHJNFQ,[music] Thank you hello everyone! Wonderful! W...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...","{'publishedAt': '2023-02-28T23:25:02Z', 'chann...","{'viewCount': '97761', 'likeCount': '1049', 'f...",1
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"{'publishedAt': '2023-02-27T22:44:53Z', 'chann...","{'viewCount': '100789', 'likeCount': '1168', '...",2
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"{'publishedAt': '2023-02-26T19:36:20Z', 'chann...","{'viewCount': '84078', 'likeCount': '759', 'fa...",3
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,"{'publishedAt': '2023-02-24T00:34:44Z', 'chann...","{'viewCount': '90645', 'likeCount': '990', 'fa...",4


### Retrieval followed by Re-ranking

#### Index construction

In [ ]:
# def id2details(df, I, column):
#     """Returns the video details based on the video id."""
#     return [list(df[df.id_value == idx][column]) for idx in I[0]]

# # Convert summaries to vectors
# model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
# embeddings = model.encode(df.summary.to_list(), show_progress_bar=True)
# print(f'Shape of the summary vector: {embeddings[0].shape, type(embeddings)}')

# # Step 1: Change data type
# embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# # Step 2: Instantiate the index
# index = faiss.IndexFlatL2(embeddings.shape[1])

# # Step 3: Pass the index to IndexIDMap
# index = faiss.IndexIDMap(index)

# # Step 4: Add vectors and their IDs
# index.add_with_ids(embeddings, df.id_value.values)

# print(f"Number of vectors in the Faiss index: {index.ntotal}")

# # Search
# user_query = "gordon ramsay delicious food"
# D, I = vector_search([user_query], model, index, num_results=10)
# print(f'L2 distance: {D.flatten().tolist()}\n\nMAG video IDs: {I.flatten().tolist()}')
# id2details(df, I, 'summary')

In [ ]:
class faiss_index:
    def __init__(self, data, model, name):
        self.data = data
        self.model = model
        self.name = name
    
    def index(self):
        encoded_data = self.model.encode(self.data['summary'].values.tolist())
        encoded_data = np.asarray(encoded_data.astype('float32'))
        self.index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
        self.index.add_with_ids(encoded_data, df['id_value'].values)

        # Serialise index and store it as a pickle
        with open(f"{project_dir}/models/{self.name}.pickle", "wb") as h:
            pickle.dump(faiss.serialize_index(self.index), h)
        faiss.write_index(self.index, f"{project_dir}/models/{self.name}.index")
        
    def fetch(self, idx):
        info = self.data.iloc[idx]
        meta_dict = {}
        meta_dict['summary'] = info['summary']
        return meta_dict

    def search(self, query, top_k):
        t=time.time()
        query_vector = self.model.encode([query])
        top_k = self.index.search(query_vector, top_k)
        print('>>>> Results in Total Time: {}'.format(time.time()-t))
        top_k_ids = top_k[1].tolist()[0]
        top_k_ids = list(np.unique(top_k_ids))
        results =  [self.fetch(idx) for idx in top_k_ids]
        return results

In [ ]:
embedder = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
faiss_idx = faiss_index(df, embedder, 'faiss_msmarco-distilbert_index')
faiss_idx.index()
results = faiss_idx.search("massive storm in the united states", 5)
print(results)

>>>> Results in Total Time: 0.021813392639160156
[{'summary': " A massive winter storm is threatening the U.S. Coast to Coast all the way from California and the West to Maine in the east. CNN's Miguel Marquez is in East Palestine with more a toxic mess left by its Durrell train now President Joe Biden is adding his pressure on Norfolk Southern following criticism for his response to the crash on Instagram. The EPA has ordered Norfolk Southern Rail Company to cover all of the cleanup costs but the rail company is facing further pressure. China's foreign Ministry calls the whole thing an accident. The U.S. says this was no weather balloon deflating. China is sticking to its weather balloon claim dismissed by the Pentagon as nothing but hot air. CNN's searched for the truth behind the balloon took our team from the Chinese Capital to Shanxi Province about 300 miles Southwest of Beijing."}, {'summary': ' The Federal Aviation Administration issued a ground stop yesterday of all domestic pl

In [ ]:
# Bi-Encoder for encoding video summary
# bi_encoder = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
top_k_count = 25

# Cross-encoder for re-ranking the retrieved results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2')

passages = df['summary'].values.tolist()
print("Video Summaries:", len(np.unique(df['id'])), len(df['id']))

Video Summaries: 435 435


#### Search

In [ ]:
def search(query, index, model, top_k_count):
    # Semantic Search
    # Encoding the query using the bi-encoder and finding potentially relevant summaries
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k_count)
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    print('>>>> Results in Total Time: {}'.format(time.time()-t))

    # Re-Ranking Results
    # Scoring all retrieved summaries with the cross_encoder
    t=time.time()
    cross_inp = [[query, passages[hit]] for hit in top_k_ids]

    bienc_op = [passages[hit] for hit in top_k_ids]
    cross_scores = cross_encoder.predict(cross_inp)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))

    # Output of top-k hits from bi-encoder
    print("\n-------------------------\n")
    print(f"Top-{top_k_count} Bi-Encoder Retrieval hits")
    results =  [passages[idx] for idx in top_k_ids]
    for result in results:
        print("\t{}".format(result.replace("\n", " ")))
        
    # Output of top-k hits from re-ranker
    print("\n-------------------------\n")
    print(f"Top-{top_k_count} Cross-Encoder Re-ranker hits")

    cross_ranks = np.argsort(np.array(cross_scores))[::-1]
    for hit in cross_ranks:
        print("\t{}".format(bienc_op[hit].replace("\n", " ")))

    cross_results = [bienc_op[hit] for hit in cross_ranks]
    return cross_scores, cross_results

#### Inference

In [ ]:
# Retrieve results based on query
query = 'massive storm in the united states'
index = faiss.read_index(f"{project_dir}/models/faiss_msmarco-distilbert_index.index")
search(query, index, embedder, top_k_count)

>>>> Results in Total Time: 0.016315221786499023
>>>> Results in Total Time: 0.02750396728515625

-------------------------

Top-25 Bi-Encoder Retrieval hits
	 Massive storm system pummeling much of the country from California to Maine. More than 65 million people across 29. States are under winter weather alerts warning of heavy snow, severe icing, extreme cold. Up next, we're taking a trip to see some penguins in need of some help. Learn something new every day on the show. African penguins made nests in boroughs of guano, layers of accumulated seabird and bad feces. In the 19th century humans began to exploit guano supplies and sell it as fertilizer, leaving penguins and their eggs exposed to predators and a scorching sun. In 2016, the African Penguin Nest project dived to the rescue to create artificial nests that give penguin parents a safe and cool place to raise their chicks.
	 A massive winter storm is threatening the U.S. Coast to Coast all the way from California and the West

(array([  3.7436466,   2.8087213, -11.385459 ,  -8.632236 , -11.089748 ,
        -11.520358 , -11.41357  , -10.905931 ,  -9.331146 , -11.287453 ,
        -11.557667 , -10.1533985,  -7.960017 , -11.54244  , -11.254047 ,
         -7.407221 , -11.275513 , -11.068649 , -11.529196 ,  -3.21132  ,
        -11.249069 , -11.486104 , -11.384414 ,  -4.98317  , -11.319244 ],
       dtype=float32),
 [" Massive storm system pummeling much of the country from California to Maine. More than 65 million people across 29. States are under winter weather alerts warning of heavy snow, severe icing, extreme cold. Up next, we're taking a trip to see some penguins in need of some help. Learn something new every day on the show. African penguins made nests in boroughs of guano, layers of accumulated seabird and bad feces. In the 19th century humans began to exploit guano supplies and sell it as fertilizer, leaving penguins and their eggs exposed to predators and a scorching sun. In 2016, the African Penguin Ne

### Finetuning the Encoder

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-base-v1') #change to large and generate synthetic data fully
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-base-v1')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
# Synthetic query generation
# Parameters
batch_size = 16
num_queries = 5 # Number of queries per summary
max_length_paragraph = 512 # Max length for summary
max_length_query = 64   # Max length for output query

def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)

# generate a sample synthetic data generation on random 100 summaries
random.seed(4)
paragraphs = random.sample(df['summary'].values.tolist(), k=100)

with open(f'{project_dir}/data/queries/generated_queries.tsv', 'w') as f_out:

    for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
        sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = _removeNonAscii(para)
            f_out.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3722: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 7/7 [00:22<00:00,  3.27s/it]


In [ ]:
query, para

('why did congress pass the debt ceiling?',
 " The debt ceiling is the legal limit set by Congress on how much money America can borrow because the U.S. spends more than it takes in it borrows. Every once in a while Congress needs to raise it and the Treasury Department won't be able to pay all the country's bills in full and on time. Dr Sanjay Gupta gives us some tips on how to find our happy place amid the cold and winter. Jakarta is sinking into the ocean at alarming rates due to excessive groundwater extraction. The government of Indonesia has a plan to build a new capital city called Nusuntara out of the dense jungles of Eastern Borneo at an estimated cost of more than 30 billion dollars. New Centaura is expected to be completed in 2045 and the government says it will be three and a half times the size of Singapore.")

In [ ]:
train_examples = [] 
with open(f'{project_dir}/data/queries/generated_queries.tsv') as f_input:
    for line in f_input:
        try:
            query, paragraph = line.strip().split('\t', maxsplit=1)
            train_examples.append(InputExample(texts=[query, paragraph]))
        except:
            pass
        
random.shuffle(train_examples)
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=8)

word_emb = models.Transformer('sentence-transformers/msmarco-distilbert-base-dot-prod-v3')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])
train_loss = losses.MultipleNegativesRankingLoss(model)

# Finetuning the model
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)
model.save(f'{project_dir}/models/finetuned_model')

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

In [ ]:
# Index construction after finetuning
faiss_ix = faiss_index(df, model, 'finetuned_faiss_index')
faiss_ix.index()

In [ ]:
# Searching with finetuned model
model_path = f'{project_dir}/models/finetuned_model'
bi_encoder = SentenceTransformer(model_path)
top_k_count = 25

# Cross-encoder for re-ranking the retrieved results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2')

passages = df['summary'].values.tolist()
print("Video Summaries:", len(passages))

Video Summaries: 435


In [ ]:
# Retrieve results based on query
query = 'gordon ramsay delicious food'
index = faiss.read_index(f"{project_dir}/models/finetuned_faiss_index.index")
search(query, index, bi_encoder, top_k_count=25)

>>>> Results in Total Time: 0.08123087882995605
>>>> Results in Total Time: 0.024992942810058594

-------------------------

Top-25 Bi-Encoder Retrieval hits
	 Murray's an MRE is a packaged meal that's usually given to soldiers in combat because they're easy to store and they last forever and you. And it contains all of the tools for cooking in the wilderness and we're going to see if they're disgusting. Here are scrambled eggs with cheese western style and pizza slice pepperoni. Orange Time: Three: It's like if you compressed all the orange Fanta in the world into a tiny cube and then ate it. Add 2 to 4 ounces of water to pouch, stir, and consume promptly. It tastes like a popsicle. It tastes super fresh little. I mean he's like it hasn't been freeze-dried in a bag for like 20 years. I can hear the crunch I'm really tall your God Are you sure it's better than dying? It's it's as good as they could possibly do with under the circumstances. It's just a fact of food life. We're going to 

(array([ -9.440701  ,  -9.51284   ,   3.352921  , -11.065905  ,
         -3.0254526 ,  -9.71814   ,  -5.995535  ,  -9.255742  ,
          2.4445667 ,   0.29215723,  -4.531797  ,  -9.724435  ,
         -3.627644  ,   2.789486  ,  -6.9951773 , -10.8980255 ,
         -9.927401  , -11.148537  , -11.14984   ,  -4.6138215 ,
         -5.3013153 ,  -3.308768  ,  -4.5448046 ,  -0.5227378 ,
          3.788906  ], dtype=float32),
 [' "Ramsay in 10" is the challenge Queen from Hell\'s Kitchen. Mia Castro makes her version of a Tailgate turkey burger with an avocado spread and some potato sticks for a little bit of crunch too. Mia says she prefers turkey burgers to beef burgers and has a list of ingredients that can be pre-made. "Ramsay in 10" is Gordon Ramsay\'s 10th episode of his cooking show. The show is a weekly weekly show on Ramsay\'s YouTube channel. Don\'t forget to subscribe to Gordon\'s channel and leave a comment below! Please share your thoughts and videos of the show with us on our bl